In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [17]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ ~s', Q = ~env(s, a)
    ################################################ ~s', Q = ~env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    #################################################### s'', Q' = ~env(s', ~a')
    next_states = tf.nn.tanh(next_states_logits)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # min
    return actions_logits, aloss, eloss, aloss2

In [18]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [19]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.dones, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [20]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [21]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [22]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [23]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
# gamma = 0.99                   # future reward discount

In [24]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [25]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                #rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              #model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:0.7048 eloss:3.3174 aloss2:2.8746 exploreP:0.9984
Episode:1 meanR:21.5000 R:27.0000 rate:0.0540 aloss:0.7055 eloss:3.2478 aloss2:2.9540 exploreP:0.9958
Episode:2 meanR:21.0000 R:20.0000 rate:0.0400 aloss:0.7022 eloss:3.2901 aloss2:2.9005 exploreP:0.9938
Episode:3 meanR:19.2500 R:14.0000 rate:0.0280 aloss:0.7010 eloss:3.2771 aloss2:2.9174 exploreP:0.9924
Episode:4 meanR:20.8000 R:27.0000 rate:0.0540 aloss:0.7024 eloss:3.2529 aloss2:2.9339 exploreP:0.9898
Episode:5 meanR:20.0000 R:16.0000 rate:0.0320 aloss:0.7007 eloss:3.2455 aloss2:2.9326 exploreP:0.9882
Episode:6 meanR:21.0000 R:27.0000 rate:0.0540 aloss:0.6997 eloss:3.2472 aloss2:2.9395 exploreP:0.9856
Episode:7 meanR:20.2500 R:15.0000 rate:0.0300 aloss:0.6964 eloss:3.2043 aloss2:2.9876 exploreP:0.9841
Episode:8 meanR:20.0000 R:18.0000 rate:0.0360 aloss:0.6975 eloss:3.2012 aloss2:2.9812 exploreP:0.9823
Episode:9 meanR:21.3000 R:33.0000 rate:0.0660 aloss:0.7003 eloss:3.1816 aloss2:3.0

Episode:80 meanR:24.3580 R:86.0000 rate:0.1720 aloss:0.6742 eloss:1.7150 aloss2:5.9160 exploreP:0.8227
Episode:81 meanR:24.8415 R:64.0000 rate:0.1280 aloss:0.6750 eloss:1.6694 aloss2:6.1181 exploreP:0.8175
Episode:82 meanR:24.9880 R:37.0000 rate:0.0740 aloss:0.6719 eloss:1.6490 aloss2:6.1471 exploreP:0.8146
Episode:83 meanR:24.9048 R:18.0000 rate:0.0360 aloss:0.6718 eloss:1.6275 aloss2:6.3803 exploreP:0.8131
Episode:84 meanR:25.0471 R:37.0000 rate:0.0740 aloss:0.6756 eloss:1.6262 aloss2:6.3233 exploreP:0.8102
Episode:85 meanR:25.1279 R:32.0000 rate:0.0640 aloss:0.6730 eloss:1.6041 aloss2:6.5152 exploreP:0.8076
Episode:86 meanR:25.2184 R:33.0000 rate:0.0660 aloss:0.6741 eloss:1.6006 aloss2:6.4871 exploreP:0.8050
Episode:87 meanR:25.3977 R:41.0000 rate:0.0820 aloss:0.6753 eloss:1.5764 aloss2:6.5793 exploreP:0.8017
Episode:88 meanR:25.3596 R:22.0000 rate:0.0440 aloss:0.6682 eloss:1.5671 aloss2:6.6839 exploreP:0.8000
Episode:89 meanR:25.3000 R:20.0000 rate:0.0400 aloss:0.6724 eloss:1.5381 

Episode:159 meanR:39.3500 R:27.0000 rate:0.0540 aloss:0.6792 eloss:1.3173 aloss2:10.7869 exploreP:0.5961
Episode:160 meanR:39.6900 R:64.0000 rate:0.1280 aloss:0.6791 eloss:1.3096 aloss2:10.7985 exploreP:0.5924
Episode:161 meanR:39.8200 R:26.0000 rate:0.0520 aloss:0.6795 eloss:1.3191 aloss2:10.7050 exploreP:0.5909
Episode:162 meanR:39.8400 R:13.0000 rate:0.0260 aloss:0.6799 eloss:1.3233 aloss2:11.0580 exploreP:0.5901
Episode:163 meanR:40.2400 R:79.0000 rate:0.1580 aloss:0.6804 eloss:1.3004 aloss2:10.8925 exploreP:0.5855
Episode:164 meanR:40.3400 R:29.0000 rate:0.0580 aloss:0.6745 eloss:1.2947 aloss2:10.8869 exploreP:0.5839
Episode:165 meanR:41.1800 R:101.0000 rate:0.2020 aloss:0.6757 eloss:1.3137 aloss2:10.8755 exploreP:0.5781
Episode:166 meanR:41.4500 R:58.0000 rate:0.1160 aloss:0.6741 eloss:1.2920 aloss2:10.8089 exploreP:0.5748
Episode:167 meanR:41.8400 R:63.0000 rate:0.1260 aloss:0.6763 eloss:1.3321 aloss2:10.8275 exploreP:0.5713
Episode:168 meanR:41.8500 R:16.0000 rate:0.0320 aloss:

Episode:237 meanR:69.8400 R:64.0000 rate:0.1280 aloss:0.6933 eloss:1.3277 aloss2:11.1392 exploreP:0.3278
Episode:238 meanR:70.0500 R:35.0000 rate:0.0700 aloss:0.7020 eloss:1.2588 aloss2:11.1117 exploreP:0.3267
Episode:239 meanR:70.4800 R:68.0000 rate:0.1360 aloss:0.7006 eloss:1.3099 aloss2:11.1112 exploreP:0.3246
Episode:240 meanR:70.6900 R:49.0000 rate:0.0980 aloss:0.7010 eloss:1.3155 aloss2:11.2817 exploreP:0.3230
Episode:241 meanR:71.2800 R:69.0000 rate:0.1380 aloss:0.7069 eloss:1.3237 aloss2:11.1232 exploreP:0.3209
Episode:242 meanR:71.7200 R:59.0000 rate:0.1180 aloss:0.6846 eloss:1.3123 aloss2:11.1209 exploreP:0.3191
Episode:243 meanR:71.6400 R:62.0000 rate:0.1240 aloss:0.6914 eloss:1.2721 aloss2:11.1237 exploreP:0.3171
Episode:244 meanR:71.9500 R:123.0000 rate:0.2460 aloss:0.6937 eloss:1.3017 aloss2:11.1465 exploreP:0.3134
Episode:245 meanR:71.6400 R:44.0000 rate:0.0880 aloss:0.6926 eloss:1.2820 aloss2:11.1861 exploreP:0.3121
Episode:246 meanR:72.4800 R:105.0000 rate:0.2100 aloss

Episode:315 meanR:50.6800 R:23.0000 rate:0.0460 aloss:0.6984 eloss:1.3567 aloss2:11.1781 exploreP:0.2372
Episode:316 meanR:50.5600 R:49.0000 rate:0.0980 aloss:0.6915 eloss:1.3238 aloss2:11.0797 exploreP:0.2361
Episode:317 meanR:50.0500 R:17.0000 rate:0.0340 aloss:0.6865 eloss:1.3312 aloss2:11.1750 exploreP:0.2357
Episode:318 meanR:49.7500 R:26.0000 rate:0.0520 aloss:0.7038 eloss:1.2847 aloss2:11.1251 exploreP:0.2351
Episode:319 meanR:48.6400 R:27.0000 rate:0.0540 aloss:0.6837 eloss:1.3151 aloss2:11.0866 exploreP:0.2345
Episode:320 meanR:47.5700 R:11.0000 rate:0.0220 aloss:0.6777 eloss:1.3513 aloss2:11.1539 exploreP:0.2343
Episode:321 meanR:47.2700 R:22.0000 rate:0.0440 aloss:0.6848 eloss:1.3256 aloss2:11.1345 exploreP:0.2338
Episode:322 meanR:47.4900 R:41.0000 rate:0.0820 aloss:0.6910 eloss:1.2822 aloss2:11.0947 exploreP:0.2329
Episode:323 meanR:46.6600 R:45.0000 rate:0.0900 aloss:0.6943 eloss:1.3494 aloss2:11.2254 exploreP:0.2319
Episode:324 meanR:46.3000 R:23.0000 rate:0.0460 aloss:0

...

Episode:551 meanR:50.0800 R:10.0000 rate:0.0200 aloss:0.8087 eloss:1.2965 aloss2:10.9722 exploreP:0.1097
Episode:552 meanR:49.0700 R:8.0000 rate:0.0160 aloss:0.7415 eloss:1.3306 aloss2:10.7169 exploreP:0.1096
Episode:553 meanR:48.6700 R:10.0000 rate:0.0200 aloss:0.8121 eloss:1.4180 aloss2:10.7316 exploreP:0.1095
Episode:554 meanR:48.0200 R:22.0000 rate:0.0440 aloss:0.7446 eloss:1.3613 aloss2:11.0104 exploreP:0.1093
Episode:555 meanR:46.9300 R:10.0000 rate:0.0200 aloss:0.7376 eloss:1.4106 aloss2:11.1439 exploreP:0.1092
Episode:556 meanR:46.2000 R:8.0000 rate:0.0160 aloss:0.7665 eloss:1.4024 aloss2:11.0980 exploreP:0.1091
Episode:557 meanR:45.4500 R:9.0000 rate:0.0180 aloss:0.7782 eloss:1.6129 aloss2:11.3466 exploreP:0.1090
Episode:558 meanR:43.8600 R:11.0000 rate:0.0220 aloss:0.7506 eloss:1.3445 aloss2:10.7214 exploreP:0.1089
Episode:559 meanR:43.2300 R:12.0000 rate:0.0240 aloss:0.7697 eloss:1.2506 aloss2:10.7832 exploreP:0.1088
Episode:560 meanR:42.5400 R:15.0000 rate:0.0300 aloss:0.76

Episode:630 meanR:14.6900 R:22.0000 rate:0.0440 aloss:0.8019 eloss:1.3519 aloss2:10.9809 exploreP:0.0983
Episode:631 meanR:14.7400 R:15.0000 rate:0.0300 aloss:0.7637 eloss:1.3198 aloss2:11.1991 exploreP:0.0982
Episode:632 meanR:14.5900 R:10.0000 rate:0.0200 aloss:0.7840 eloss:1.3638 aloss2:10.6989 exploreP:0.0981
Episode:633 meanR:14.4900 R:10.0000 rate:0.0200 aloss:0.7658 eloss:1.2549 aloss2:10.8399 exploreP:0.0980
Episode:634 meanR:14.4300 R:9.0000 rate:0.0180 aloss:0.7531 eloss:1.2598 aloss2:10.8923 exploreP:0.0980
Episode:635 meanR:14.3300 R:11.0000 rate:0.0220 aloss:0.7763 eloss:1.3742 aloss2:10.9464 exploreP:0.0979
Episode:636 meanR:14.3400 R:10.0000 rate:0.0200 aloss:0.7825 eloss:1.2595 aloss2:10.6671 exploreP:0.0978
Episode:637 meanR:14.3300 R:11.0000 rate:0.0220 aloss:0.7966 eloss:1.3242 aloss2:11.2370 exploreP:0.0977
Episode:638 meanR:14.3500 R:10.0000 rate:0.0200 aloss:0.7079 eloss:1.2482 aloss2:11.0238 exploreP:0.0976
Episode:639 meanR:14.3600 R:9.0000 rate:0.0180 aloss:0.8

Episode:709 meanR:18.1600 R:26.0000 rate:0.0520 aloss:0.7372 eloss:1.3346 aloss2:11.0767 exploreP:0.0865
Episode:710 meanR:18.1600 R:15.0000 rate:0.0300 aloss:0.7482 eloss:1.2490 aloss2:11.0574 exploreP:0.0864
Episode:711 meanR:18.3100 R:35.0000 rate:0.0700 aloss:0.7479 eloss:1.3415 aloss2:11.0286 exploreP:0.0861
Episode:712 meanR:18.1700 R:10.0000 rate:0.0200 aloss:0.8390 eloss:1.3521 aloss2:11.4043 exploreP:0.0860
Episode:713 meanR:18.1900 R:30.0000 rate:0.0600 aloss:0.7841 eloss:1.3437 aloss2:11.0892 exploreP:0.0858
Episode:714 meanR:18.2400 R:23.0000 rate:0.0460 aloss:0.7847 eloss:1.4918 aloss2:11.1379 exploreP:0.0856
Episode:715 meanR:18.2900 R:22.0000 rate:0.0440 aloss:0.7628 eloss:1.3206 aloss2:10.8748 exploreP:0.0855
Episode:716 meanR:18.1300 R:11.0000 rate:0.0220 aloss:0.7750 eloss:1.5160 aloss2:11.2475 exploreP:0.0854
Episode:717 meanR:18.3100 R:28.0000 rate:0.0560 aloss:0.7470 eloss:1.3962 aloss2:10.8688 exploreP:0.0852
Episode:718 meanR:18.2100 R:10.0000 rate:0.0200 aloss:0

Episode:788 meanR:26.4100 R:28.0000 rate:0.0560 aloss:0.7855 eloss:1.4145 aloss2:11.1359 exploreP:0.0715
Episode:789 meanR:26.5700 R:43.0000 rate:0.0860 aloss:0.7967 eloss:1.3591 aloss2:11.0517 exploreP:0.0712
Episode:790 meanR:26.6400 R:27.0000 rate:0.0540 aloss:0.7696 eloss:1.3670 aloss2:11.1742 exploreP:0.0710
Episode:791 meanR:26.6700 R:30.0000 rate:0.0600 aloss:0.7751 eloss:1.2809 aloss2:10.9282 exploreP:0.0709
Episode:792 meanR:26.9300 R:46.0000 rate:0.0920 aloss:0.7630 eloss:1.4410 aloss2:11.0269 exploreP:0.0706
Episode:793 meanR:27.0600 R:37.0000 rate:0.0740 aloss:0.7877 eloss:1.3474 aloss2:10.8659 exploreP:0.0704
Episode:794 meanR:27.2700 R:31.0000 rate:0.0620 aloss:0.7993 eloss:1.3440 aloss2:10.8796 exploreP:0.0702
Episode:795 meanR:27.4200 R:32.0000 rate:0.0640 aloss:0.7716 eloss:1.3642 aloss2:10.8066 exploreP:0.0700
Episode:796 meanR:27.5400 R:37.0000 rate:0.0740 aloss:0.7924 eloss:1.4105 aloss2:11.0746 exploreP:0.0698
Episode:797 meanR:27.5800 R:32.0000 rate:0.0640 aloss:0

Episode:867 meanR:36.5000 R:49.0000 rate:0.0980 aloss:0.7433 eloss:1.3011 aloss2:11.1029 exploreP:0.0557
Episode:868 meanR:36.5800 R:34.0000 rate:0.0680 aloss:0.7888 eloss:1.4567 aloss2:11.2698 exploreP:0.0555
Episode:869 meanR:36.6800 R:40.0000 rate:0.0800 aloss:0.7540 eloss:1.2927 aloss2:11.0629 exploreP:0.0554
Episode:870 meanR:36.6800 R:32.0000 rate:0.0640 aloss:0.7749 eloss:1.2965 aloss2:11.2310 exploreP:0.0552
Episode:871 meanR:36.6700 R:45.0000 rate:0.0900 aloss:0.7896 eloss:1.3388 aloss2:11.1637 exploreP:0.0550
Episode:872 meanR:36.7100 R:43.0000 rate:0.0860 aloss:0.7653 eloss:1.3294 aloss2:11.0945 exploreP:0.0548
Episode:873 meanR:36.8700 R:44.0000 rate:0.0880 aloss:0.7432 eloss:1.3889 aloss2:11.1087 exploreP:0.0546
Episode:874 meanR:36.8600 R:33.0000 rate:0.0660 aloss:0.8086 eloss:1.3851 aloss2:11.1446 exploreP:0.0545
Episode:875 meanR:36.9900 R:48.0000 rate:0.0960 aloss:0.7679 eloss:1.3848 aloss2:10.9237 exploreP:0.0543
Episode:876 meanR:37.0700 R:39.0000 rate:0.0780 aloss:0

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [39]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
